https://github.com/thomaspingel/intro_to_geospatial_python/blob/master/10.%20Open%20Source%20Geospatial%20Tools%20for%20Python.ipynb

So far we've used a number of packages for image input and output, with imageio and matplotlib likely to be most commonly used.  For image processing, we've looked at scipy's ndimage and scikit-image, both of which can apply filters to perform neighborhood calculations.  Now we'll look at geospatial-specific libraries to do similar things.

# GDAL

[GDAL](https://en.wikipedia.org/wiki/GDAL) ([gdal.org](https://gdal.org/)) is the Geospatial Data abstraction library, and it's the core engine for reading raster geospatial data.  GDAL is used in many applications: ArcGIS and QGIS both include distributions, as do many Python packages.  You might try to do a search for GDAL.exe in your program files and see what comes up.

GDAL is written in C, but includes Python bindings.  The Python Bindings for GDAL are [here](https://gdal.org/api/python_bindings.html).

Installation is usually done with conda (or with conda-forge), but [Chrisoph Gohlke's Unofficial Precompiled Python Libraries](https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal) still come in handy once in a while.  I usually use Rasterio these days for geospatial image manipulation and that installs gdal as a dependency.

It's nice to know a little GDAL, though, because it has fewer dependencies and will likely be a part of the geospatial computing landscape for decades to come.  

Remember that GDAL is a nice collection of commandline tools as well, [each of which can be run with Python](https://stackoverflow.com/questions/89228/how-do-i-execute-a-program-or-call-a-system-command).  Two very valuable ones are [GDALWARP](https://gdal.org/programs/gdalwarp.html), which can project rasters, and [GDALDEM](https://gdal.org/programs/gdaldem.html) which can calculate many morphometric variables.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

from osgeo import gdal    # This used to be a simple import gdal

In [ ]:
I = imageio.v2.imread('data/nasadem_blacksburg_area.tif')

In [ ]:
I

In [23]:
# This is one way to read through a GeoTIFF using GDAL.
# The data is read, but also the geotransform and the projection.  Other metadata is available as well.
# At the end, the object (src) can be deleted to close it.  This is safer than the .close() command.

src = gdal.Open('data/nasadem_blacksburg_area.tif')
image_array = src.ReadAsArray()
gt = src.GetGeoTransform()
projection = src.GetProjection()
del src

In [26]:
# It would be better to use a "with" command, but this doesn't seem to work

with gdal.Open('data/nasadem_blacksburg_area.tif') as src:
    image_array = src.ReadAsArray()
    gt = src.GetGeoTransform()
    projection = src.GetProjection()

AttributeError: __enter__

In [24]:
# The image_array is just what you'd expect, and is no different than what you'd get with an imageio read.

image_array

array([[607.5784 , 608.8551 , 609.8437 , ..., 641.1217 , 646.8145 ,
        652.50726],
       [607.0702 , 608.2813 , 609.0966 , ..., 643.54565, 648.70953,
        654.4122 ],
       [607.4888 , 608.10114, 608.35315, ..., 646.5481 , 650.92975,
        656.6323 ],
       ...,
       [548.7557 , 548.9494 , 548.2284 , ..., 699.3221 , 696.8954 ,
        695.38135],
       [546.67267, 546.6051 , 545.77203, ..., 696.76276, 692.43494,
        689.51666],
       [545.5378 , 545.44525, 544.2103 , ..., 695.25116, 689.15027,
        684.99023]], dtype=float32)

In [25]:
# The projection data is a standard string:

projection

'PROJCS["WGS 84 / UTM zone 17N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32617"]]'

In [27]:
# The geotransform is a six-digit tuple containing standard coordinate referencing information: 
# x, resolution_x, rotation_x, y, rotation_y, resolution_y

gt

(537546.6982884794,
 17.48589996167486,
 0.0,
 4130330.7579188175,
 0.0,
 -17.48589996167495)

## [Iowa State NEXRAD data](https://mesonet.agron.iastate.edu/docs/nexrad_mosaic/) and [GDAL GeoTransforms](https://gdal.org/tutorials/geotransforms_tut.html)

In [29]:
# Here is an example with a PNG and associated world file
# https://mesonet.agron.iastate.edu/docs/nexrad_mosaic/
# Even though the world file is not a part of the PNG, GDAL knows to look for it due to naming conventions.

# Open the image; I is an object, not an array!
image_object = gdal.Open('data/n0q_202304190000.png')

# Read the array data
image_array = image_object.ReadAsArray()

# Read the transformation array
gt = image_object.GetGeoTransform()

# Close the image object when you're done
del image_object

In [30]:
# Look at the geotransform; this provides information about grid size and location.  
# Compare to World File!
# Unlike the world file, the tuple shown here references from the upper left corner (not the center) of the upper left pixel

print(gt)

(-126.0025, 0.005, 0.0, 50.0025, 0.0, -0.005)


Take a moment to review GDAL's [GeoTransform Tutorial](https://gdal.org/tutorials/geotransforms_tut.html) as this affects how pixel coordinates are calculated.  GDAL imagines that 0,0 in pixel space corresponds to the upper left corner of the upper left pixel.  Some conceptions of pixel space imagine that 0,0 is the center of that pixel.  You must be careful when processing raster data to handle this correctly!  For an extended discussion, see [ETOPO1's grid/node vs cell/pixel registration of raster](https://www.ncei.noaa.gov/products/grid-cell-registration). 

<img src="http://www.ncei.noaa.gov/sites/default/files/2020-04/gridregistrationfigure1.PNG" width=600>

So this image is 5400 x 12200, with a cell size of .005 (degrees) and the upper left corner of the upper left pixel is at -126.0025 (longitude) and 50.0025 (latitude)

We can do some simple math to get the center of the upper left corner pixel (We'll calculate pixel centers, not edges)

In [32]:
gt

(-126.0025, 0.005, 0.0, 50.0025, 0.0, -0.005)

In [31]:
ul_corner = gt[0] + gt[1]/2, gt[3] + gt[5]/2
print('Upper left corner:',ul_corner)

Upper left corner: (-126.0, 50.0)


In [34]:
# Can you calculate the center of the lower right pixel?

nrows,ncols = np.shape(image_array)

lr_corner = gt[0] + ncols*gt[1] + gt[1]/2, gt[3] + nrows*gt[5] + gt[5]/2
print('Lower right corner:',lr_corner)

Lower right corner: (-65.0, 22.999999999999996)


In [35]:
# We can calculate all pixel centers this way

lons = np.linspace(gt[0],lr_corner[0],ncols)
lats = np.linspace(gt[3],lr_corner[1],nrows)

In [36]:
print(lats)

[50.0025     49.99749861 49.99249722 ... 23.01000278 23.00500139
 23.        ]


## Automatically downloading data

In [37]:
import time
import datetime
import os
import urllib
import sys

In [41]:
url = 'https://mesonet.agron.iastate.edu/archive/data/2023/04/19/GIS/uscomp/n0q_202304190045.png'
urllib.request.urlretrieve(url,'myfile.png')

('myfile.png', <http.client.HTTPMessage at 0x2a34dfca2e0>)

In [42]:
# We could write a function to download NEXRAD data between some given dates
# There's a lot of code here, but most of the work is being done by urllib.request.urlretrieve

import time
import datetime
import os
import urllib
import sys

def fetch_iastate(start_date,end_date,outdir,request_delay=.01,output=False):
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    current_date = start_date  
    time_delta = datetime.timedelta(minutes=5)
    while current_date < end_date:
        image_path = outdir + '/' + current_date.strftime('%Y%m%d%H%M.png')
        if output:
                print("checking for " + image_path)
        if not os.path.exists(image_path):
            url = current_date.strftime('http://mesonet.agron.iastate.edu/archive/data/%Y/%m/%d/GIS/uscomp/n0r_%Y%m%d%H%M.png')
            if output:
                print('Retreiving ' + current_date.strftime('%Y%m%d%H%M.png'))
            try:
                urllib.request.urlretrieve(url,image_path)
            except:
                if output:
                    print('Failed to retrieve.')
        world_path = outdir + '/' + current_date.strftime('%Y%m%d%H%M.wld')
        if not os.path.exists(world_path):
            url = current_date.strftime('http://mesonet.agron.iastate.edu/archive/data/%Y/%m/%d/GIS/uscomp/n0r_%Y%m%d%H%M.wld')
            if output:
                print('Retreiving ' + current_date.strftime('%Y%m%d%H%M.wld'))
            try:
                urllib.request.urlretrieve(url,world_path)
            except:
                if output:
                    print('Failed to retrieve.')
        current_date += time_delta
        time.sleep(request_delay)
    sys.stdout.flush()

In [44]:
# And then download it.  WARNING: Don't download too much data, unless you've got a good reason!  It's not nice to tie
# up resources.  This may also take a while.  We delay some amount of time to get images to avoid swamping the server
# and there are a lot of images per day (12 per hour * 24 hours per day = 288 images per day).
# 
# Overall, using an FTP client or wget would be more effective (that's its job, after all) 
# but it's good to know how it's done.

# I'll set it to download a day's work of data.
start_date = datetime.datetime(2017,10,11)
end_date = datetime.datetime(2017,10,12) 
outdir = 'nexrad'

fetch_iastate(start_date,end_date,outdir,output=True,request_delay=1)
print('Done.')

checking for nexrad/201710110000.png
Retreiving 201710110000.png
Retreiving 201710110000.wld
checking for nexrad/201710110005.png
Retreiving 201710110005.png
Retreiving 201710110005.wld
checking for nexrad/201710110010.png
Retreiving 201710110010.png
Retreiving 201710110010.wld
checking for nexrad/201710110015.png
Retreiving 201710110015.png
Retreiving 201710110015.wld
checking for nexrad/201710110020.png
Retreiving 201710110020.png
Retreiving 201710110020.wld
checking for nexrad/201710110025.png
Retreiving 201710110025.png
Retreiving 201710110025.wld
checking for nexrad/201710110030.png
Retreiving 201710110030.png
Retreiving 201710110030.wld
checking for nexrad/201710110035.png
Retreiving 201710110035.png
Retreiving 201710110035.wld
checking for nexrad/201710110040.png
Retreiving 201710110040.png
Retreiving 201710110040.wld
checking for nexrad/201710110045.png
Retreiving 201710110045.png
Retreiving 201710110045.wld
checking for nexrad/201710110050.png
Retreiving 201710110050.png
Retre

KeyboardInterrupt: 

# [Rasterio](https://rasterio.readthedocs.io/en/stable/intro.html)

I don't often use GDAL to handle rasters.  Why not?  GDAL's documentation is not always spectacular, and it can be difficult to find Python examples.  Mapbox developed Rasterio to "wrap" GDAL, and provide a better API.  I typically install Rasterio with conda when I setup an environment.

In [45]:
import rasterio

In [46]:
# Rasterio follows GDAL syntax (unfortunately!), so you open a file and then read the contents:

src = rasterio.open('data/nasadem_blacksburg_area.tif')
metadata = src.profile
image_array = src.read()
src.close()  # Don't forget to close it!

# Is it closed?
src.closed

True

In [47]:
# Rasterio DOES support with-syntax, so I recommend you use it.

with rasterio.open('data/nasadem_blacksburg_area.tif') as src:
    metadata = src.profile
    image_array = src.read()

In [48]:
# The image array is now-familiar

image_array

array([[[607.5784 , 608.8551 , 609.8437 , ..., 641.1217 , 646.8145 ,
         652.50726],
        [607.0702 , 608.2813 , 609.0966 , ..., 643.54565, 648.70953,
         654.4122 ],
        [607.4888 , 608.10114, 608.35315, ..., 646.5481 , 650.92975,
         656.6323 ],
        ...,
        [548.7557 , 548.9494 , 548.2284 , ..., 699.3221 , 696.8954 ,
         695.38135],
        [546.67267, 546.6051 , 545.77203, ..., 696.76276, 692.43494,
         689.51666],
        [545.5378 , 545.44525, 544.2103 , ..., 695.25116, 689.15027,
         684.99023]]], dtype=float32)

In [49]:
# The metadata dictionary contains many useful pieces as a dictionary.  Note that data type, nodata value, width,
# height, band count, CRS, transform* (more in a second), etc.  And because you can load this without loading the image
# data, any call here is quite fast since you're just reading header data.

metadata

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 3.3999999521443642e+38, 'width': 2007, 'height': 1438, 'count': 1, 'crs': CRS.from_epsg(32617), 'transform': Affine(17.48589996167486, 0.0, 537546.6982884794,
       0.0, -17.48589996167495, 4130330.7579188175), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}

## Transforms are the key to the whole operation

The transform object is actually an [Affine matrix](https://en.wikipedia.org/wiki/Affine_transformation) the governs simple translations, rotations, and scaling.  It is a way to use [linear algebra](https://en.wikipedia.org/wiki/Linear_algebra) to quite efficiently transform coordinates from one space to another.  In this case, from pixel space to geographic coordinates.

References:
* I have more on [this subject](https://github.com/thomaspingel/geography_single_shot_notebooks/blob/master/2D%20Rigid%20Rotations%2C%20Translations%2C%20and%20Matrix%20Multiplication.ipynb) in one of my [geography_single_shot_notebooks](https://github.com/thomaspingel/geography_single_shot_notebooks).
* The Python Affine package gives a [great overview](https://pypi.org/project/affine/).

More:
* http://www.willamette.edu/~gorr/classes/GeneralGraphics/Transforms/transforms2d.htm
* https://subscription.packtpub.com/book/data/9781789537147/1/ch01lvl1sec04/applying-affine-transformation

To transform in 2D space, we need a 3x3 matrix, but the bottom row is not needed, so we really only need six parameters.  This is where the Affine matrix comes from.  For 3D coordinate transforms, we need a 4x4 matrix (but again the bottom row is not really needed).  The crux of the structure looks like this:

<img src="https://static.packt-cdn.com/products/9781789537147/graphics/assets/4510565b-9462-4a22-ae17-fbc7b85876d1.png" width=350>

In [72]:
# Let's pull the IAstate data again, since it's very simple and has a worldfile we can inspect:

with rasterio.open('data/n0q_202304190000.png') as src:
    metadata = src.profile
    gt = src.transform          # You can read this directly, or pull metadata['transform']
    image_array = src.read()

In [74]:
image_array = np.squeeze(image_array)

In [75]:
np.shape(image_array)

(5400, 12200)

In [51]:
# Let's pull that transform.  Notice its similarity to the data we got from GDAL.  Same numbers, slightly different
# order.  Affine has a .from_gdal() method should you need to convert.

# Here's the Affine transform from Rasterio:
gt

Affine(0.005, 0.0, -126.0025,
       0.0, -0.005, 50.0025)

In [52]:
# You can print (or convert) the geotransform as a matrix, but be careful when units are small, they can hide
# the true precision:

print(gt)

| 0.01, 0.00,-126.00|
| 0.00,-0.01, 50.00|
| 0.00, 0.00, 1.00|


 The earlier geotransform from GDAL looked like this:

(-126.0025, 0.005, 0.0, 50.0025, 0.0, -0.005)

And the world file looks like this:
~~~
0.005
0.0
0.0
-0.005
-126.0
50.0
~~~

In [53]:
gt

Affine(0.005, 0.0, -126.0025,
       0.0, -0.005, 50.0025)

In [54]:
# The affine matrix is a simple mathemetical matrix that allows coordinate transformation like so:

# suppose you wanted the coordinate of the upper left pixel (0,0):

col, row = 0,0
gt * (col,row)

# Note carefully that this is the upper left CORNER of the pixel, which you can confirm against the world file.
# Just as before, 0,0 starts at the very left-most edge of the raster.  The center is at .5,.5

(-126.0025, 50.0025)

In [55]:
# Center of upper left pixel, same as world file specification.

col, row = .5, .5
gt * (col,row)

(-126.0, 50.0)

In [56]:
# We can generate 20 random rows, cols, and calculate those geocoords as well.

# Note carefully that rows are your Y coord and cols are your X coord.  You must be
# mindful of the requested / expected order of inputs and outputs.  They are not 
# always what you would expect.  In the US, it's often easy to pick out lat from lon
# since lon is negative, but for other coord systems and locations you will need
# to be careful.

k = 20
rows, cols = np.random.randint(0,metadata['height']-1,k), np.random.randint(0,metadata['width']-1,k)
gt * (cols,rows)

(array([-119.9925, -117.2525, -102.2575, -124.6875, -117.1075, -124.9375,
         -75.8075,  -96.6525,  -71.4625, -100.8175,  -93.0625,  -83.0475,
        -117.5725, -107.4575,  -95.7825,  -85.9725,  -85.7325, -102.5275,
        -102.5225, -114.9325]),
 array([36.5275, 33.0125, 30.2725, 38.9725, 24.7075, 42.1975, 49.7625,
        38.5275, 47.0025, 26.8925, 28.2925, 35.5675, 26.0375, 40.4475,
        49.4675, 42.6575, 39.0975, 43.0225, 42.9275, 44.5025]))

In [57]:
# Inverse operations are also quite easy.  The tilda operator returns the inverse of a matrix:

~gt

Affine(200.0, 0.0, 25200.5,
       0.0, -200.0, 10000.5)

In [58]:
# And this is used to convert geog coords to pixel coords.  This is the center of the upper left pixel
# as seen in the world file.

~gt * (-126.0,50.0)

(0.5, 0.5)

In [59]:
# Apart from this, rasterio has a function to do the conversion for you (although the straight math is so easy!)

map_x, map_y = rasterio.transform.xy(gt,rows,cols,offset='ul')
print(map_x)
print(map_y)

[-119.99249999999999, -117.2525, -102.2575, -124.6875, -117.1075, -124.9375, -75.8075, -96.6525, -71.4625, -100.8175, -93.0625, -83.0475, -117.57249999999999, -107.4575, -95.7825, -85.9725, -85.73249999999999, -102.5275, -102.5225, -114.9325]
[36.527499999999996, 33.012499999999996, 30.272499999999997, 38.9725, 24.707499999999996, 42.1975, 49.762499999999996, 38.527499999999996, 47.0025, 26.8925, 28.292499999999997, 35.567499999999995, 26.037499999999998, 40.4475, 49.4675, 42.6575, 39.0975, 43.022499999999994, 42.927499999999995, 44.5025]


In [60]:
# Not everyone uses the upper-left corner is zero and the center of that pixel is .5,.5 convention.  So you can 
# apply an offset with map, or with rasterio.transform.xy:

# If you want 0,0 to refer to the CENTER of the upper left pixel (as I often do!), do this:

rasterio.transform.xy(gt,0,0,offset='center')

(-126.0, 50.0)

In [64]:
np.shape(image_array)

(1, 5400, 12200)

In [80]:
image_array = np.array([[45,11],[23,12]])
image_array

array([[45, 11],
       [23, 12]])

In [81]:
image_array[0,0]

45

## Writing images with rasterio

In [ ]:
# Let's load a DEM

with rasterio.open('data/nasadem_blacksburg_area.tif') as src:
    metadata = src.profile
    Z = src.read()              # You may want to read a specific band!

In [ ]:
# The resolution is here, or you could extract from the Affine matrix in the geotransform

src.res

In [ ]:
# Pixels are not always square!  Define the resolution as the average of these:

res = np.mean(src.res)
res

In [ ]:
# You would think plt.imshow would work.  Why doesn't it?

plt.imshow(Z)

In [ ]:
# The problem is the shape.  The shape gets reported as (1,1438,2007).  Why the extra coordinate?
# Because this is the first band.  You can fix it by reading in only the bands you want.
# Or, better yet, you can get rid of the extra dimension using .squeeze:

np.shape(np.squeeze(Z))

In [ ]:
Z = np.squeeze(Z)

In [ ]:
# Now calculate slope for it:
res = np.mean(src.res)

gy,gx = np.gradient(Z,res)
S = np.sqrt(gx**2 + gy**2)   # Rise over run
S = np.arctan(S)             # Radians
S = np.rad2deg(S)            # Degrees

plt.imshow(S)

In [ ]:
# Suppose you want to write this data out.  How would you do it?
# Because you're using the same metadata as before, a simple write like this is easy:

fn = 'slope.tif'
with rasterio.open(fn, 'w', **metadata) as dst:
    dst.write(S, 1)                                   # Write to band 1

In [ ]:
# If things have changed or you're authoring your own content,
# you can control the metadata as individual parameters directly:

with rasterio.open(
    'slope2',
    'w',
    driver='GTiff',
    height=S.shape[0],
    width=S.shape[1],
    count=1,
    dtype=S.dtype,
    crs=metadata['crs']                  ,            # Or whatever this should be
    transform=metadata['transform'],                  # or whatever this should be
) as dst:
    dst.write(S, 1)

# Multiband data

In [ ]:
# Let's download some NAIP imagery

url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/NAIP/va_2016/38078/m_3807808_ne_17_1_20160718_20160928.jp2'

with rasterio.open(url) as src:
    metadata = src.profile
    I = src.read()              # You may want to read a specific band!

In [ ]:
# How many bands?

src.count

<img src="https://www.researchgate.net/publication/342413913/figure/fig2/AS:905930921226240@1593002171931/Formula-used-to-calculate-the-normalized-difference-vegetation-index-NDVI_W640.jpg" width="200">

In [ ]:
# Calculate NDVI:

I = I.astype(float)
ndvi = (I[3,:,:] - I[0,:,:]) / (I[3,:,:] + I[0,:,:])

plt.imshow(ndvi,cmap='turbo')

## Sample some data

This requires that the source is open

In [ ]:
url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/NAIP/va_2016/38078/m_3807808_ne_17_1_20160718_20160928.jp2'

src = rasterio.open(url)

In [ ]:
# make some random data in map coordinates

k = 10
x = src.width * np.random.rand(k) + src.bounds.left
y = src.height * np.random.rand(k) + src.bounds.bottom

print(x,y)

In [ ]:
# Sample the values at those coordinates

result = [x for x in src.sample(zip(x,y))]

In [ ]:
# The result is an array of values, one "column" for band and "row" for each 
result

# There are ways to do this without the src.sample method.  Ask me how!

# PyProj

PyProj lets you transform coordinates from one CRS to another, among other things.  It's quite easy to use!

In [ ]:
from pyproj import Transformer

def coord_transform(x,y,from_epsg,to_epsg):
    transformer = Transformer.from_crs(from_epsg,to_epsg,always_xy=True)
    return transformer.transform(x,y)

In [ ]:
# These are some random coords within our boundary.  What are the lat/lon of these?

x = src.width * np.random.rand(k) + src.bounds.left
y = src.height * np.random.rand(k) + src.bounds.bottom

In [ ]:
src.crs

In [ ]:
coord_transform(x,y,src.crs,4326)

## What else can I do with Rasterio?

So, so much!  [Look through the documentation](https://rasterio.readthedocs.io/en/stable/) to discover more.

# Assignment

Write a Jupyter notebook with three sections:
1. Use rasterio to georeference an image of your choosing (not geospatial) by using rasterio.from_origion to create a transform, an epsg code to define your coordinate system (anything is fine, 4326 is WGS84), and rasterio to write out the image.  Bring the image into ArcGIS Pro and include a screenshot of it overlaid on a map along with the assignment.
2. Review the rasterio documentation and explore five functions not discussed in lecture.
3. Demonstrate with your own data that you can combine a raster layer of your choice and a geopandas overlay.  It can be simple!  I might for instance choose some data from Natural Earth.